<a target="_blank" href="https://colab.research.google.com/github/Nicolepcx/Oxford_multi_agent_investment_analysis/blob/main/Oxford_langgraph_multiturn_conversation.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# About the Notebook



## 🔁 Interactive Investment Report Generator with Human-in-the-Loop Feedback

This notebook implements a LangGraph workflow that generates LinkedIn posts using an LLM, with a human-in-the-loop feedback mechanism. The flow includes three main components:

- **Model Node**: Uses a language model (Llama 3 via Nebius) to generate a LinkedIn post based on a user-provided topic and optional feedback.
- **Human Node**: Pauses execution to collect user feedback on the generated post. The user can iteratively refine the output or type `"done"` (or `"exit"`, `"quit"`, `"q"`) to finalize it.
- **End Node**: Outputs the final post and feedback summary.

The code uses LangGraph’s `interrupt()` and `and command()` mechanism to handle real-time input, and supports continuous, feedback-driven improvement of the generated content. The flow is structured using a `StateGraph`, and all interactions are managed with a `while True` loop that handles streaming and user input cleanly.

**NOTE:** You can use either Open AI or any other LLM here, I used [Nebius](https://studio.nebius.com/), which offers fairly good prices on some common LLMs.



# Dependencies

In [1]:
%%capture --no-stderr
%pip install openai==1.61.1 python-dotenv==1.0.1 langchain-openai==0.2.13 langgraph==0.3.31

# API Setup

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

# Set up API keys and environment variables
NEBIUS_API_KEY = os.getenv('NEBIUS_API_KEY')

# Uncomment to use OpenAI
#OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [21]:
# Uncomment to use OpenAI
#llm = ChatOpenAI(model="gpt-4o")

# Imports

In [22]:
from langgraph.graph import StateGraph, START, END, add_messages
from langgraph.types import Command, interrupt
from typing import TypedDict, Annotated, List
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
import uuid

# Constants

In [23]:
llm = ChatOpenAI(
                model="meta-llama/Llama-3.3-70B-Instruct-fast",
                temperature=0,
                max_tokens=None,
                timeout=None,
                max_retries=2,
                api_key=NEBIUS_API_KEY,
                base_url="https://api.studio.nebius.ai/v1/"
            )


A `StateGraph` object defines the structure of our chatbot as a "state machine". We'll add `nodes` to represent the llm and functions our chatbot can call and `edges` to specify how the bot should transition between these functions.

In [44]:
class State(TypedDict):
    report_asset: str
    generated_report: Annotated[List[str], add_messages]
    human_feedback: Annotated[List[str], add_messages]


In [45]:
def model(state: State):
    """ Here, we're using the LLM to generate a first feedback human feedback
    incorporated on the initial prompt for the report."""

    #logger("[model] Generating content")
    hypothesis_prompt = state["report_asset"]
    feedback = state["human_feedback"] if "human_feedback" in state else ["No Feedback yet"]


    # Define the prompt

    prompt = f"""

        Report Asset: {report_asset}
        Human Feedback: {feedback[-1] if feedback else "No feedback yet"}

        Generate a structured and well-written Asset report based on the given asset.

        Consider previous human feedback to refine the reponse.
    """

    response = llm.invoke([
        SystemMessage(content="You are a CFA charter investment analyst with over 20 years of experience in investing."),
        HumanMessage(content=prompt)
    ])

    generated_report = response.content

    print(f"[model_node] Generated Report:\n{generated_report}\n")

    return {
       "generated_report": [AIMessage(content=generated_report)] ,
       "human_feedback": feedback
    }

# Define Human Node Using `Interrupt()`

In [51]:
def human_node(state: State):
    """Human Intervention node - loops back to model unless input is done"""

    print("\n [human_node] awaiting human feedback...")

    generated_report = state["generated_report"]

    user_feedback = interrupt(
        {
            "generated_report": generated_report,
            "message": "Provide feedback or type 'done' to finish"
        }
    )

    print(f"[human_node] Received human feedback: {user_feedback}")

    if user_feedback.lower() in ["done", "quit", "exit", "q"]:
        return Command(update={"human_feedback": state["human_feedback"] + ["Finalised"]}, goto="end_node")

    return Command(update={"human_feedback": state["human_feedback"] + [user_feedback]}, goto="model")


In [52]:
def end_node(state: State):
    """ Final node """
    print("\n[end_node] Process finished")
    print("Final Generated Post:", state["generated_report"][-1])
    print("Final Human Feedback", state["human_feedback"])
    return {"generated_report": state["generated_report"], "human_feedback": state["human_feedback"]}


# Buiding the Graph

In [53]:
graph = StateGraph(State)
graph.add_node("model", model)
graph.add_node("human_node", human_node)
graph.add_node("end_node", end_node)

graph.set_entry_point("model")



# Define the flow

In [54]:
graph.add_edge(START, "model")
graph.add_edge("model", "human_node")

graph.set_finish_point("end_node")

# Enable Interrupt mechanism

In [55]:
checkpointer = MemorySaver()
app = graph.compile(checkpointer=checkpointer)

thread_config = {"configurable": {
    "thread_id": uuid.uuid4()
}}

report_asset = input("Enter your asset: ")
initial_state = {
    "report_asset": report_asset,
    "generated_report": [],
    "human_feedback": []
}

# Initial call to stream the first output
stream = app.stream(initial_state, config=thread_config)
feedback_done = False

while True:
    try:
        chunk = next(stream)
        for node_id, value in chunk.items():
            if node_id == "__interrupt__":
                while True:
                    user_feedback = input("User: ")
                    if user_feedback.lower() in ["quit", "exit", "q", "done"]:
                        app.invoke(Command(resume="done"), config=thread_config)
                        feedback_done = True
                        break
                    else:
                        app.invoke(Command(resume=user_feedback), config=thread_config)
                if feedback_done:
                    break
    except StopIteration:
        break

print("Finished.")






Enter your asset: nvidia
[model_node] Generated Report:
**Asset Report: NVIDIA Corporation (NVDA)**

**Introduction:**
NVIDIA Corporation is a leading American technology company specializing in the design and manufacture of graphics processing units (GPUs), high-performance computing hardware, and related software. The company is a dominant player in the fields of artificial intelligence (AI), gaming, professional visualization, and autonomous vehicles.

**Business Overview:**
NVIDIA's business is organized into two main segments: Graphics and Compute & Networking. The Graphics segment includes GeForce GPUs for gaming and Quadro GPUs for professional visualization. The Compute & Networking segment comprises Datacenter, Mellanox, and Automotive businesses, which provide hardware and software solutions for AI, high-performance computing, and networking applications.

**Financial Performance:**
As of the latest available data, NVIDIA's financial performance is characterized by:

* **Reve